In [1]:
p = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
p8 = [6, 3, 7, 4, 8, 5, 10, 9]

ip = [2, 6, 3, 1, 4, 8, 5, 7]
inv_ip = [4, 1, 3, 5, 7, 2, 8, 6]


ep = [4, 1, 2, 3, 2, 3, 4, 1]


p4 = [2, 4, 3, 1]


s0 = [["01", "00", "11", "10"],
     ["11", "10", "01", "00"],
     ["00", "10", "01", "11"],
     ["11", "01", "11", "10"]]
s1 = [["00", "01", "10", "11"],
     ["10", "00", "01", "11"],
     ["11", "00", "01", "00"],
     ["10", "01", "00", "11"]]

key = '1000110101'

In [2]:
def create_8_bit_key(key10):
    new_key = ''
    for i in p8:
        new_key += key[i-1]
    return new_key

In [3]:
def generate_keys(key):
    keyp = ''
    for i in p:
        keyp += key[i-1]
    
    left_half = keyp[1:5] + keyp[0]
    right_half = keyp[6:] + keyp[5]
    k1 = create_8_bit_key(left_half+right_half)
    
    left_half = keyp[3:5] + keyp[0:3]
    right_half = keyp[8:] + keyp[5:8]
    k2 = create_8_bit_key(left_half+right_half)
    
    return k1,k2

In [4]:
generate_keys(key)

('10001110', '10001110')

In [5]:
def generate_ip(key8):
    new_key = ''
    for i in ip:
        new_key += key8[i-1]
    return new_key

In [6]:
def generate_inv_ip(key8):
    new_key = ''
    for i in inv_ip:
        new_key += key8[i-1]
    return new_key

In [7]:
def swap(data):
    return data[4:] + data[:4]

In [8]:
def expand_and_permutate(key4):
    new_key = ''
    for i in ep:
        new_key += key4[i-1]
    return new_key

In [9]:
def perform_xor(data,k):
    size = len(data)
    data = int(data, 2)
    k = int(k, 2)
    res = data ^ k
    return format(res, 'b').zfill(size)
    

In [10]:
def get_sbox(data, sbox):
    row = int(data[0] + data[3], 2)
    col = int(data[1:3], 2)
    return sbox[row][col]

In [11]:
def permutate_4(key4):
    new_key = ''
    for i in p4:
        new_key += key4[i-1]
    return new_key

In [12]:
def fk(key8, k):
    left_key = key8[:4]
    right_key = key8[4:]
    
    right_key = expand_and_permutate(right_key)
    right_key = perform_xor(right_key, k)    
    right_key = get_sbox(right_key[:4], s0) + get_sbox(right_key[4:], s1)
    right_key = permutate_4(right_key)    
    left_key = perform_xor(left_key, right_key)

    return left_key + key8[4:]

In [13]:
def encrypt(data):
    data = generate_ip(data)
    k1, k2 = generate_keys(key)
    
    #round 1
    data = fk(data, k1)
    
    #swap left and right part
    data = swap(data)
    
    #round 2
    data = fk(data, k2)
    
    data = generate_inv_ip(data)
    
    return data

In [14]:
def decrypt(encrypted_data):
    data = generate_ip(encrypted_data)
    k1, k2 = generate_keys(key)
    
    #round 1    
    data = fk(data, k2)
    
    #swap left and right part
    data = swap(data)
    
    #round 2
    data = fk(data, k1)
    
    data = generate_inv_ip(data)
    
    return data

In [15]:
data = input("Enter data to encrypt : ")
encrypted_data = ""
for i in data:
    d = ord(i)
    d = "{0:08b}".format(d)
    encrypted_data += encrypt(d)
print("Encrypted data : ", encrypted_data)

Enter data to encrypt : Sample text
Encrypted data :  0101001110010011111011001110010110101011110101101100100001000000110101101110101101000000


In [16]:
original_data = ""
for i in range(int(len(encrypted_data)/8)):
    original_data += chr(int(decrypt(encrypted_data[int(i*8):int((i+1)*8)]), 2))
print("Decrypted Data : ", original_data)

Decrypted Data :  Sample text
